In [10]:
import torch
import torch.nn.functional as F

from torch_geometric.nn import GCNConv
from torch_geometric.datasets import KarateClub

In [15]:
dataset = KarateClub()

In [16]:
dataset.num_node_features

34

In [17]:
class Net(torch.nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [23]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()

In [26]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred.eq(data.y).sum().item())

In [27]:
correct

34.0

In [ ]:
acc = correct / data.sum().item()
print('Accuracy: {:.4f}'.format(acc))